## Graph/network theory
- In this notebook we will show you some of the common commands you can use when you study and make network graphs using Python.¶We will use a simple example with a bunch of kids who have been interacting with eachother during a school day. Some of them got sick with a virus infection later same day. We will study the network to see how if it can tell us something about the transmission of the virus among the kids.
- It is related to the presentation..... 

## Installation of libraries and necessary software¶
- Code to install libraries
- State that this has to be run only once
- If necessary, further installation instructions

In [ ]:
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import warnings
from custom import load_data as cf
import csv
from operator import itemgetter
import community 
import pandas as pd
from custom import ecdf

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Loading data¶

- Load the two csv files: 
        Kids_nodes.csv (contains names and information about whether the kid got sick or not) 
        Kids_edges.cvs (contains information about who played with whom)
        (If you would like to see the content of the two files use pd.read_csv("kids_nodes.csv") and pd.read_cs("kids_edges.csv"))

In [ ]:
# Open the file with nodes
with open('kids_nodes.csv', 'r') as nodecsv:                 
    nodereader = csv.reader(nodecsv) 
    nodes = [n for n in nodereader][1:]                     

# Get a list of only the node names                                       
node_names = [n[0] for n in nodes]    

# Open the file with edges
with open('kids_edges.csv', 'r') as edgecsv:
    edgereader = csv.reader(edgecsv) 
    edges = [tuple(e) for e in edgereader][1:] 
    
G = nx.Graph()

G.add_nodes_from(node_names)
G.add_edges_from(edges)

### Exercise 1 - Nodes and edges
Let us first look at the nodes and edges in the network that we are going to create. 

We can start by examining out how many nodes the network contains. We do so by finding the length of the list containing the nodes of the networks.
At the same time we would also like to see the names of the nodes. The list may though be very long so we can limit the number of nodes we would like to see by using [0:5] which will only return the first five nodes.

In [ ]:
# The number of nodes in the network
print(len(G.nodes()))

# The first five nodes in the network
print(list(G.nodes())[0:5])



Now, it turned out that this list was not very long. Edit the code to get the name of the rest of the nodes (excl. the names of the first five nodes)

In [ ]:
# The name of the rest of the nodes.
print(list(G.nodes())[5:11])

Now let us look at the same for the edges. The number of edges in the network is also represented by the lenght of the list of edges. So we can simply reuse the command for the length of the list of nodes and exchange the word nodes with the word edges. The same can be done for the list-commands.

In [ ]:
# The number of edges in the network
print(len(G.edges()))

# List all the edges in the network
print(list(G.edges()))

You can also ask to have the number of nodes and edges reported together (comma separated) by giving both commands in one line, seperated by comma (len_, len_). 

In [ ]:
# The number of nodes and edges in the network
print(len(G.nodes()),",",len(G.edges()))

**Add your answers here**
*(double-click here to edit the cell)*

***Question I:<u>How many kids do we have in the study?</u>***

11


***Question II: <u>Is there any kids with the name Lea?</u>***

No


***Question III: <u>Did Mette and Anna have any contact at the day of the study? </u>***

Yes

### Exercise 2 - Drawing the network
Now we know a bit about the size and nature of the network. Let's try to draw the network - without and with names on the nodes.

We do so by using the command nx.draw from the networkx package

In [ ]:
# Draw network
nx.draw(G)


The graph/network is drawn with labels on the nodes by default. You can change that by using the command se the command nx.draw(G, with_labels=True)

In [ ]:
# Draw network with labels
nx.draw(G, with_labels=True)

The two networks may not look completely alike. This is simply because the angel in which is is displayed is not the same. The overall structure (topology) of the networks are the same.

Below you can see a code that can help you substract a graph showing only the play mates of a specific kid. In the example below you will see who Anna has been playing with that day (you do not need to understand the entire code)

In [ ]:
def extract_neighbor_edges2(G, node):
    neighbors = G.neighbors(node)
    newG = nx.Graph()
    
    for neighbor in neighbors:
        if (node, neighbor) in G.edges() or (neighbor, node) in G.edges():
            newG.add_edge(node, neighbor)

    return newG

fig = plt.figure(1)
newG = extract_neighbor_edges2(G, 'anna')
nx.draw(newG, with_labels=True)

Below is a copy of the code above - modify the code to show a graph of the playmates of Mads. 

In [ ]:
def extract_neighbor_edges2(G, node):
    neighbors = G.neighbors(node)
    newG = nx.Graph()
    
    for neighbor in neighbors:
        if (node, neighbor) in G.edges() or (neighbor, node) in G.edges():
            newG.add_edge(node, neighbor)

    return newG

fig = plt.figure(1)
newG = extract_neighbor_edges2(G, 'mads')
nx.draw(newG, with_labels=True)

**Add your answers here**
*(double-click here to edit the cell)*

***Question I:<u>What is the name of the kids that Anna played with that day?</u>***

Lise, Mette og Lotte


***Question II: <u>How many kids did Mads play with that day?</u>***

7



### Exercise 3 - Extracting information about neighbors (centrality), degree of centrality and ranking based on centrality. 
We just saw how we could get subgraphs containing the playmates of a specific kid - in graph theory these will be the neighbors of a specific node. We can also make commands that will give us the information as numbers and lists. 

If we would like to know the neighbors (here playmates) of e.g. the first node (here Mads), we can use the G.neighbors(node) function.

In [ ]:
# Neighbors of the node with the name "mads"
list(G.neighbors('mads'))

The number of neighbors - i.e. the number of other nodes, that one node is connected to - is a measure of the nodes centrality. We could count them like we did above - or we could use the len function to find out the length of the list above. Let's use the len function

In [ ]:
# The number of neighbors of the node "mads"? (i.e. what is the centrality of "mads")
len(list(G.neighbors('mads')))

Now pick any kid and calculate its centrality and see who the kid played with


In [ ]:
# List of neighbors
list(G.neighbors('jesper'))

In [ ]:
# Calculation of centrality
len(list(G.neighbors('jesper')))

The next script illustrate the possibility to rank the nodes based on the their centrality. It starts with pd. indicating that it is a function from the pandas package. Here the five kids with the highest centrality ranked with the highest centrality ranked at the top. 

In [ ]:
(pd.Series({n: len(list(G.neighbors(n))) 
            for n in G.nodes()})
 .sort_values(ascending=False)
 .head(5)
)

Now try to change the copy of the code to rank all the kids based on their centrality

In [ ]:
(pd.Series({n: len(list(G.neighbors(n))) 
            for n in G.nodes()})
 .sort_values(ascending=False)
)

*(This could be done either by changing 5 to the total number of nodes or by deleting .head(5) completely)*

Sometimes we are not only interested in the number of neighbors (centrality), but also in the degree of centrality. Here you see how we can make a list of the nodes along with their degree of centrality. The list is not ranked, but listed according to their original order in the csv file. 

In [ ]:
# nx.degree_centrality(G)
list(nx.degree_centrality(G).items())[0:5]

We can also plot the degree of centrality. 

In [ ]:
# Plotting a graph with degree centralities
fig = plt.figure(0)
# Get a list of degree centrality scores for all of the 
# nodes in the graph
degree_centralities = list(
    nx.degree_centrality(G).values())
x, y = ecdf(degree_centralities)
# Plot the graph of degree centralities.
plt.scatter(x, y)
# Set the plot title. 
plt.title('Degree Centralities')

*Here the plot is given as a scatterplot - you can change the plt.scatter(x,y) to plt.hist(x,y) to get it as a histogram instead.*

**Add your answers here**
*(double-click here to edit the cell)*

***Question I:<u>What will happen if you change False to True in the code ranking the centralities?</u>***

The ranking will now be given with the one with the lowest centralities at the top.

***Question II: <u>How many of the kids only had a single playmate at the day of the study</u>***

None



### Exercise 4 - Sick or not sick?
Now, in the csv file we also have information about whether or not the kids got sick or not. Let us use this information and see if it can tell us something about the contamination.

KH: I don't know how to do so!!